
<h3>Scraping content from US National Forests webpages</h3>

<img src="images/Gimli.jpg" width=30% style="display: inline-block">
<br>
image source: lotr.wikia.com
<br>
Getting it done through whatever means possible

<br>